# Test feasibility XAI for Random Forest in pyspark

In [2]:
 import sys
 !{sys.executable} -m pip install pyspark

Could not fetch URL https://pypi.org/simple/pyspark/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pyspark/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1002)'))) - skipping
Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1002)'))) - skipping


ERROR: Could not find a version that satisfies the requirement pyspark (from versions: none)
ERROR: No matching distribution found for pyspark


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import *
from pyspark.sql.functions import weekofyear
from pyspark.sql.functions import month
from pyspark.sql.functions import sum,avg,max
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql import functions as F
from pyspark.sql.functions import lag

ModuleNotFoundError: No module named 'pyspark'

In [3]:
spark= SparkSession \
       .builder \
       .appName("XAIspark") \
       .getOrCreate()


https://www.kaggle.com/code/hellbuoy/online-retail-k-means-hierarchical-clustering/notebook

In [4]:
# import required libraries for dataframe and visualization

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

# import required libraries for clustering
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import cut_tree

In [5]:
# import sys
# !{sys.executable} -m pip install pyarrow

In [6]:
import pyspark.sql.functions as f


In [7]:
retail = spark.read.format("csv").option("header", "true").load("data/OnlineRetail.csv")

In [8]:
type(retail)

pyspark.sql.dataframe.DataFrame

In [9]:
print((retail.count(), len(retail.columns)))

(541909, 8)


## Data cleaning 

In [10]:
# Calculating the Missing Values % contribution in DF
 
# Find Count of Null, None, NaN of All DataFrame Columns
from pyspark.sql.functions import col,isnan, when, count
retail.select([count(when(isnan(c)| col(c).isNull(), c)).alias(c) for c in retail.columns]
   ).show()

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|        0|        0|       1454|       0|          0|        0|    135080|      0|
+---------+---------+-----------+--------+-----------+---------+----------+-------+



In [11]:
# Droping rows having missing values

retail = retail.dropna()
print((retail.count(), len(retail.columns)))

(406829, 8)


In [12]:
# New Attribute : Monetary

retail = retail.withColumn('Amount' , f.col('Quantity')*f.col('UnitPrice'))

rfm_m = retail.groupBy('CustomerID').agg(f.sum('Amount').alias('Amount'))

#rfm_m = rfm_m.reset_index()
rfm_m.show(5)

+----------+------------------+
|CustomerID|            Amount|
+----------+------------------+
|     16250|            389.44|
|     15574| 702.2500000000002|
|     15555| 4758.200000000002|
|     15271|2485.8199999999997|
|     17714|             153.0|
+----------+------------------+
only showing top 5 rows



In [13]:
# New Attribute : Frequency
rfm_f = retail.groupBy('CustomerID').agg(f.count('InvoiceNo').alias('Frequency'))

#rfm_f = rfm_f.reset_index()
rfm_f.select('CustomerID', 'Frequency').show(5)
rfm_f = rfm_f.select('CustomerID', 'Frequency')

+----------+---------+
|CustomerID|Frequency|
+----------+---------+
|     16250|       24|
|     15574|      168|
|     15555|      925|
|     15271|      275|
|     17714|       10|
+----------+---------+
only showing top 5 rows



In [14]:
# Merging the two dfs

rfm = rfm_m.join(rfm_f, on='CustomerID', how='inner')
rfm.show(5)

+----------+------------------+---------+
|CustomerID|            Amount|Frequency|
+----------+------------------+---------+
|     16250|            389.44|       24|
|     15574| 702.2500000000002|      168|
|     15555| 4758.200000000002|      925|
|     15271|2485.8199999999997|      275|
|     17714|             153.0|       10|
+----------+------------------+---------+
only showing top 5 rows



In [15]:
# New Attribute : Recency

# Convert to datetime to proper datatype
retail = retail.withColumn("InvoiceDate", to_timestamp(f.col('InvoiceDate'), 'dd-MM-yyyy HH:mm'))

retail.show(5)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+------------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|            Amount|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+------------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|     17850|United Kingdom|15.299999999999999|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|     17850|United Kingdom|             20.34|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|     17850|United Kingdom|              22.0|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|     17850|United Kingdom|             20.34|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|     17850|United Kingdom|    

In [16]:
# Compute the maximum date to know the last transaction date

max_date = retail.agg(f.max('InvoiceDate')).take(1)[0][0]
max_date

datetime.datetime(2011, 12, 9, 12, 50)

In [17]:
# Compute the difference between max date and transaction date

retail = (retail
          .withColumn('max_date' , f.lit(max_date))
          .withColumn('Diff' ,  f.datediff(f.col('max_date'), f.col('InvoiceDate'))))

retail.select('InvoiceDate' , 'max_date' , 'Diff').show(5)

+-------------------+-------------------+----+
|        InvoiceDate|           max_date|Diff|
+-------------------+-------------------+----+
|2010-12-01 08:26:00|2011-12-09 12:50:00| 373|
|2010-12-01 08:26:00|2011-12-09 12:50:00| 373|
|2010-12-01 08:26:00|2011-12-09 12:50:00| 373|
|2010-12-01 08:26:00|2011-12-09 12:50:00| 373|
|2010-12-01 08:26:00|2011-12-09 12:50:00| 373|
+-------------------+-------------------+----+
only showing top 5 rows



In [18]:
# Compute last transaction date to get the recency of customers
rfm_p = (retail
         .groupby('CustomerID')
         .agg(f.min('Diff').alias('Recency'))
        )
rfm_p.show(5)

+----------+-------+
|CustomerID|Recency|
+----------+-------+
|     16250|    261|
|     15574|    177|
|     15555|     12|
|     15271|      7|
|     17714|    320|
+----------+-------+
only showing top 5 rows



In [19]:
# Merge tha dataframes to get the final RFM dataframe

rfm = rfm.join(rfm_p, on='CustomerID', how='inner')
rfm = rfm.select('CustomerID', 'Amount', 'Frequency', 'Recency')
rfm.show(5)

+----------+------------------+---------+-------+
|CustomerID|            Amount|Frequency|Recency|
+----------+------------------+---------+-------+
|     16250|            389.44|       24|    261|
|     15574| 702.2500000000002|      168|    177|
|     15555| 4758.200000000002|      925|     12|
|     15271|2485.8199999999997|      275|      7|
|     17714|             153.0|       10|    320|
+----------+------------------+---------+-------+
only showing top 5 rows



## Handling Outliers

In [20]:
quantiles = rfm.approxQuantile("Amount", [0.05, 0.95], 0)
quantiles

[101.10000000000001, 5639.15]

In [21]:
# Removing (statistical) outliers for Amount
Q1 = quantiles[0]
Q3 = quantiles[1]
IQR = Q3 - Q1

rfm = rfm.filter((f.col('Amount') >= Q1 - 1.5*IQR ) & (f.col('Amount') <= Q3 + 1.5*IQR ) ) 

print((rfm.count(), len(rfm.columns)))

(4314, 4)


In [22]:
# Removing (statistical) outliers for Amount
quantiles = rfm.approxQuantile("Amount", [0.05, 0.95], 0)
Q1 = quantiles[0]
Q3 = quantiles[1]
IQR = Q3 - Q1

rfm = rfm.filter((f.col('Amount') >= Q1 - 1.5*IQR ) & (f.col('Amount') <= Q3 + 1.5*IQR ) ) 

print((rfm.count(), len(rfm.columns)))
# Removing (statistical) outliers for Recency
quantiles = rfm.approxQuantile("Recency", [0.05, 0.95], 0)
Q1 = quantiles[0]
Q3 = quantiles[1]
IQR = Q3 - Q1

rfm = rfm.filter((f.col('Recency') >= Q1 - 1.5*IQR ) & (f.col('Recency') <= Q3 + 1.5*IQR ) ) 
print((rfm.count(), len(rfm.columns)))

# Removing (statistical) outliers for Frequency
quantiles = rfm.approxQuantile("Frequency", [0.05, 0.95], 0)
Q1 = quantiles[0]
Q3 = quantiles[1]
IQR = Q3 - Q1

rfm = rfm.filter((f.col('Frequency') >= Q1 - 1.5*IQR ) & (f.col('Frequency') <= Q3 + 1.5*IQR ) ) 
print((rfm.count(), len(rfm.columns)))

(4301, 4)
(4301, 4)
(4281, 4)


In [23]:
feature_cols = ['Amount', 'Frequency', 'Recency']

## Vector Assembling

In [24]:
#vector assembler 
from pyspark.ml.feature import VectorAssembler

assemble       = VectorAssembler(inputCols= feature_cols, outputCol='features', handleInvalid="keep")
assembled_data = assemble.transform(rfm)

In [25]:
assembled_data.show(5)

+----------+------------------+---------+-------+--------------------+
|CustomerID|            Amount|Frequency|Recency|            features|
+----------+------------------+---------+-------+--------------------+
|     16250|            389.44|       24|    261| [389.44,24.0,261.0]|
|     15574| 702.2500000000002|      168|    177|[702.250000000000...|
|     15271|2485.8199999999997|      275|      7|[2485.81999999999...|
|     17714|             153.0|       10|    320|  [153.0,10.0,320.0]|
|     17551|            306.84|       43|    359| [306.84,43.0,359.0]|
+----------+------------------+---------+-------+--------------------+
only showing top 5 rows



## Rescaling the attributes

In [26]:
from pyspark.ml.feature import StandardScaler
scale= StandardScaler(inputCol='features',outputCol='Scaled_features')  #save this and load 
data_scale=scale.fit(assembled_data)
data_scale_output=data_scale.transform(assembled_data)

In [27]:
data_scale_output.show(5)

+----------+------------------+---------+-------+--------------------+--------------------+
|CustomerID|            Amount|Frequency|Recency|            features|     Scaled_features|
+----------+------------------+---------+-------+--------------------+--------------------+
|     16250|            389.44|       24|    261| [389.44,24.0,261.0]|[0.23658248201602...|
|     15574| 702.2500000000002|      168|    177|[702.250000000000...|[0.42661269514110...|
|     15271|2485.8199999999997|      275|      7|[2485.81999999999...|[1.51012085416256...|
|     17714|             153.0|       10|    320|  [153.0,10.0,320.0]|[0.09294658932942...|
|     17551|            306.84|       43|    359| [306.84,43.0,359.0]|[0.18640347365909...|
+----------+------------------+---------+-------+--------------------+--------------------+
only showing top 5 rows



## Building the Model - Unsupervised learning - KMeans model

In [28]:
from pyspark.ml.clustering import KMeans
kmeans=KMeans(featuresCol = 'Scaled_features' , k = 8)
knn_model = kmeans.fit(data_scale_output.select('Scaled_features'))

In [29]:
transformed = knn_model.transform(data_scale_output)

In [30]:
transformed.show(5)

+----------+------------------+---------+-------+--------------------+--------------------+----------+
|CustomerID|            Amount|Frequency|Recency|            features|     Scaled_features|prediction|
+----------+------------------+---------+-------+--------------------+--------------------+----------+
|     16250|            389.44|       24|    261| [389.44,24.0,261.0]|[0.23658248201602...|         1|
|     15574| 702.2500000000002|      168|    177|[702.250000000000...|[0.42661269514110...|         2|
|     15271|2485.8199999999997|      275|      7|[2485.81999999999...|[1.51012085416256...|         3|
|     17714|             153.0|       10|    320|  [153.0,10.0,320.0]|[0.09294658932942...|         1|
|     17551|            306.84|       43|    359| [306.84,43.0,359.0]|[0.18640347365909...|         1|
+----------+------------------+---------+-------+--------------------+--------------------+----------+
only showing top 5 rows



In [31]:
summary = (transformed.groupBy("prediction")
            .agg(f.countDistinct("CustomerID").alias("count"),
                 f.avg("Amount").alias("Amount"),
                 f.avg("Recency").alias("Recency"),
                 f.avg("Frequency").alias("Frequency")               
                )
            .withColumn('client %',( f.col('count')/f.sum('count').over(Window.partitionBy()) )*100   )
            .withColumn('sales %',( f.col('Amount')/f.sum('Amount').over(Window.partitionBy()) )*100  )  
          )
    
summary.show()

+----------+-----+------------------+------------------+------------------+------------------+------------------+
|prediction|count|            Amount|           Recency|         Frequency|          client %|           sales %|
+----------+-----+------------------+------------------+------------------+------------------+------------------+
|         1|  465| 303.4993763440862|313.94408602150537| 20.86021505376344|10.861948142957253|1.5394658151960157|
|         6|   88| 4441.849431818182|15.363636363636363|502.67045454545456|2.0555944872693295|22.530772349197214|
|         3|  318| 3378.879465408807|25.852201257861637|238.13836477987422|7.4281709880868965|17.138978976903598|
|         5|  108| 8048.664166666671|26.787037037037038|226.03703703703704| 2.522775052557814| 40.82592686625105|
|         4|  729|1925.5212359396435|   33.039780521262|116.90534979423869| 17.02873160476524| 9.766985866232481|
|         7|  730| 578.8339616438357| 92.91095890410959|30.876712328767123|17.0520906330

## Building the Model - Supervised learning -Random forest model

In [32]:

final_data =(transformed
             .withColumnRenamed("prediction", "Cluster_ID")
             .select('Scaled_features','Cluster_ID')
            )

In [33]:
#Splitting train and test data
train, test = final_data.randomSplit([0.75,0.25], seed = 25)

In [34]:
from pyspark.ml.classification import RandomForestClassifier
rf  = RandomForestClassifier(featuresCol = 'Scaled_features', labelCol = 'Cluster_ID', maxDepth = 10)
rfModel = rf.fit(train)
predictions = rfModel.transform(test)

## Evaluation

In [35]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol = "Cluster_ID", predictionCol = "prediction" , metricName = "accuracy")
evaluator.evaluate(predictions)

0.9420702754036088

In [36]:
from pyspark.ml.evaluation import ClusteringEvaluator
evaluator2 = ClusteringEvaluator( featuresCol = "Scaled_features", predictionCol = "prediction" ,metricName = "silhouette")
evaluator2.evaluate(predictions)

0.4265000020292632

In [37]:
# Silhouette analysis
range_n_clusters = [2, 3, 4, 5, 6, 7, 8]

for num_clusters in range_n_clusters:
    # intialise kmeans
    kmeans=KMeans(featuresCol = 'Scaled_features' , k = num_clusters)
    knn_model = kmeans.fit(data_scale_output.select('Scaled_features'))
    transformed = knn_model.transform(data_scale_output)
   
    
    # silhouette score
    evaluator2 = ClusteringEvaluator( featuresCol = "Scaled_features", predictionCol = "prediction" ,metricName = "silhouette")
    silhouette_avg = evaluator2.evaluate(transformed)
    print("For n_clusters={0}, the silhouette score is {1}".format(num_clusters, silhouette_avg))
    

For n_clusters=2, the silhouette score is 0.44455610706328524
For n_clusters=3, the silhouette score is 0.6825137557251696
For n_clusters=4, the silhouette score is 0.6461862440589253
For n_clusters=5, the silhouette score is 0.5642148290421279
For n_clusters=6, the silhouette score is 0.5573833641026827
For n_clusters=7, the silhouette score is 0.5769959381636159
For n_clusters=8, the silhouette score is 0.4674976976048298


## XAI

In [38]:
import sys
!{sys.executable} -m pip install dalex

In [39]:
import dalex as dx 

In [96]:
model = knn_model 
X = transformed.select(feature_cols)
y =( transformed
    .select('prediction')
    .withColumnRenamed("prediction", "Cluster_ID")
    .select('Cluster_ID')
   )

In [71]:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [98]:
X_numpy = np.array(X.collect())
y_numpy = np.array(y.collect())

In [73]:
X_numpy.shape

(4281, 3)

In [78]:
import pandas as pd

In [99]:
X_pandas = pd.DataFrame(X_numpy, columns= feature_cols)
y_pandas = pd.DataFrame(y_numpy, columns= ['prediction'])

In [86]:
type(X_pandas)

pandas.core.frame.DataFrame

In [100]:
 isinstance(y_pandas, pd.DataFrame)

True

In [101]:
exp = dx.Explainer(model, X_pandas, y_pandas)

Preparation of a new explainer is initiated

  -> data              : 4281 rows 3 cols
  -> target variable   : Parameter 'y' was a pandas.DataFrame. Converted to a numpy.ndarray.
  -> target variable   : 4281 values
  -> model_class       : pyspark.ml.clustering.KMeansModel (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_default at 0x000002DE4301E9E0> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> model type        : regression will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         :  'residual_function' returns an Error when executed:
An error occurred while calling z:org.apache.spark.ml.python.MLSerDe.loads.
: net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for numpy.dtype). This happens when an unsupported/unregistered class i

C:\Users\isharau\AppData\Local\anaconda3\lib\site-packages\dalex\_explainer\object.py:138: UserWarning: 
  -> predicted values  : 'predict_function' returns an Error when executed: 
An error occurred while calling z:org.apache.spark.ml.python.MLSerDe.loads.
: net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for numpy.dtype). This happens when an unsupported/unregistered class is being unpickled that requires construction arguments. Fix it by registering a custom IObjectConstructor for this class.
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:759)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:199)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:109)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:122)
	at org.apache.spark.mllib.api.python.SerDeBase.loads(PythonMLLibAPI.scala:1326)
	at org.apache.spark.ml.pyt

In [93]:
test_pandas = test.toPandas()
exp.predict(test_pandas)

C:\Users\isharau\AppData\Local\anaconda3\lib\site-packages\pyspark\sql\pandas\conversion.py:117: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: VectorUDT()
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)


Py4JJavaError: An error occurred while calling z:org.apache.spark.ml.python.MLSerDe.loads.
: net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for numpy.core.multiarray._reconstruct). This happens when an unsupported/unregistered class is being unpickled that requires construction arguments. Fix it by registering a custom IObjectConstructor for this class.
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:759)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:199)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:109)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:122)
	at org.apache.spark.mllib.api.python.SerDeBase.loads(PythonMLLibAPI.scala:1326)
	at org.apache.spark.ml.python.MLSerDe.loads(MLSerDe.scala)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorIm

In [ ]:
obs_bd = exp.predict_parts(obs, type='break_down')

In [ ]:
obs_bd.result

In [ ]:
obs_bd.plot()